In [0]:
# --------------------------------------------
# 📌 SECTION 0 — Install 2 Python TS forecasting libraries
# --------------------------------------------

%pip install pmdarima statsmodels

# pmdarima: Automates ARIMA model selection for time series analysis.
# statsmodels: Provides statistical models, including manual ARIMA fitting.


In [0]:
# --------------------------------------------
# 📌 SECTION 1 — Load Data from Databricks
# --------------------------------------------

# It queries the train_imputed_timebins_lags table using Spark, selects relevant columns 
# (timestamp, country, grid_stress_score, etc.), orders them by time, 
# and converts the result to a Pandas DataFrame for modeling and visualization.

print("="*80)
print("SECTION 1 — LOAD DATA FROM DATABRICKS")
print("="*80)

from pyspark.sql import functions as F
import pandas as pd

# Load from your existing table
df_spark = spark.table("workspace.default.train_imputed_timebins_lags")

df_spark = df_spark.select(
    "timestamp", "country", "grid_stress_score",
    "mean_temperature_c", "Actual_Load"
)

df_spark = df_spark.orderBy("timestamp")

df = df_spark.toPandas()
df["timestamp"] = pd.to_datetime(df["timestamp"])

print(df.head())
print(df.shape)


In [0]:
# --------------------------------------------
# 📌 SECTION 2 — Prepare TS & Fit ARIMA per country
# --------------------------------------------

# It loops through each country in the dataset, prepares its time series data (hourly stress scores), 
# and fits an ARIMA model using auto_arima. 
# The result is a dictionary of fitted models — one per country — ready for forecasting.


print("="*80)
print("SECTION 2 — PREPARE TIME SERIES (PER COUNTRY)")
print("="*80)

from pmdarima import auto_arima

countries = sorted(df["country"].unique())
print("Countries found:", countries)

arima_models = {}   # to store fitted models
series_info = {}    # optional: to store basic info about each TS

for country in countries:
    print("\n" + "-"*80)
    print(f"▶ Training ARIMA for country: {country}")
    print("-"*80)

    # Filter this country's data
    df_country = df[df["country"] == country].copy()

    # Sort & index by timestamp
    df_country = df_country.sort_values("timestamp")
    df_country = df_country.set_index("timestamp")

    # Univariate time series (grid stress)
    ts = df_country["grid_stress_score"]

    print("Head:")
    print(ts.head())
    print("Length:", len(ts))

    # Skip if too few points
    if len(ts) < 30:
        print("⚠️ Not enough observations, skipping.")
        continue

    # Fit ARIMA automatically
    model = auto_arima(
        ts,
        seasonal=False,         # set True + m=24 or 7 if you want seasonality
        trace=False,
        suppress_warnings=True,
        stepwise=True
    )

    arima_models[country] = model
    series_info[country] = {"n_obs": len(ts)}

    print(f"✔ Fitted ARIMA for {country}, order={model.order}")


In [0]:
# --------------------------------------------
# 📌 SECTION 3 — Forecast with ARIMA models
# --------------------------------------------

# It generates 6-hour-ahead forecasts from the ARIMA models for each country 
# and stores the results in a dictionary (arima_forecasts) for later use or visualization.


print("="*80)
print("SECTION 3 — FORECAST WITH ARIMA (PER COUNTRY)")
print("="*80)

arima_forecasts = {}

n_periods = 6  # forecast horizon

for country, model in arima_models.items():
    print(f"\n▶ Forecasting for {country}")
    fc = model.predict(n_periods=n_periods)
    arima_forecasts[country] = fc
    print(fc[:5])



In [0]:
# --------------------------------------------
# 📌 SECTION 4 — SAVE ARIMA MODELS FOR STREAMLIT
# --------------------------------------------
print("="*80)
print("SECTION 4 — SAVE ARIMA MODELS")
print("="*80)

import os
import pickle

# SAME directory where you saved XGBoost & features
output_dir = "/Workspace/Users/(enter your account Email for Databricks here)"
os.makedirs(output_dir, exist_ok=True)

# Create subfolder for ARIMA models
arima_dir = f"{output_dir}/arima_models"
os.makedirs(arima_dir, exist_ok=True)

# Save model for each country
for country, model in arima_models.items():
    model_path = f"{arima_dir}/arima_{country}.pkl"
    
    with open(model_path, "wb") as f:
        pickle.dump(model, f)

    print(f"✔ Saved ARIMA model for {country} → {model_path}")


